In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook was created following the instructions provided on the medium post created by Gonzalo Gasca Meza available here: https://medium.com/@gogasca_/ai-platform-notebooks-with-voila-c3c57d4e8e

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
PROJECT_ID = '[CHANGE-ME]'
INSTANCE_NAME = 'voila-' + TIMESTAMP
SERVICE_ACCOUNT = INSTANCE_NAME + '-sa'
LOCATION = 'us-central1-a'


_SERVICE_ACCOUNT_ID = f'{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com'
_SERVICE_ACCOUNT_MEMBER = f"serviceAccount:{_SERVICE_ACCOUNT_ID}"



In [ ]:
# Reference: https://cloud.google.com/iam/docs/creating-managing-service-accounts

! gcloud iam service-accounts create $SERVICE_ACCOUNT \
    --description="SA for notebook with Voila" \
    --display-name=$SERVICE_ACCOUNT


In [ ]:
# Reference: https://cloud.google.com/vertex-ai/docs/general/access-control#basic-roles & https://cloud.google.com/iam/docs/creating-managing-service-accounts

! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=$_SERVICE_ACCOUNT_MEMBER \
    --role="roles/aiplatform.admin"

## Add the user that will need access to the notebook. You may need to repeat this step multiple times

In [ ]:
# Reference: https://cloud.google.com/iam/docs/creating-managing-service-accounts
AUTHORIZED_USER_EMAIL = 'email@domain.com'

_AUTHORIZED_USER_MEMBER = f'user:{AUTHORIZED_USER_EMAIL}'


! gcloud iam service-accounts add-iam-policy-binding \
    $_SERVICE_ACCOUNT_ID \
    --member=$_AUTHORIZED_USER_MEMBER \
    --role="roles/iam.serviceAccountUser"

## Create the Notebook Instance

In [ ]:
# Reference: https://cloud.google.com/vertex-ai/docs/general/troubleshooting#opening_a_notebook_results_in_a_403_forbidden_error

! gcloud notebooks instances create $INSTANCE_NAME \
  --vm-image-project=deeplearning-platform-release \
  --vm-image-family=tf2-latest-cpu \
  --service-account=$_SERVICE_ACCOUNT_ID \
  --location=$LOCATION

## Instructions to install Voila 

1) Go to your GCP Console and find the recently created USER-MANAGED NOTEBOOK instance: [link](https://pantheon.corp.google.com/vertex-ai/workbench/list/instances) (make sure you're on the right project)
2) Open JupyterLab
3) Press cancel on the pop-up message
4) Open a new terminal tab inside JupyterLab
6) Install the following libraries

`conda install -y -c conda-forge ipywidgets ipyvolume bqplot scipy`

`conda install -y -c conda-forge voila`

7) Close the JupyterLab tab 
8) Continue below


## Reset the instance

In [ ]:
! gcloud beta notebooks instances reset $INSTANCE_NAME --location=$LOCATION

## Renders a sample notebook

In [ ]:
var = ! gcloud beta notebooks instances describe $INSTANCE_NAME --location=$LOCATION
var = str(var)

p = [x for x in var.split(',') if 'proxyUri' in x]
p = p[0].replace('"', "").replace("'", '')
uri = p.split(':')[-1].strip()

url = f"https://{uri}/voila/render/tutorials/README.ipynb"


from IPython.display import display, HTML

# create a string template for the HTML snippet
link_t = f"<h2><a href='{url}'> Click here to open a sample notebook</a></h2>"

# create HTML object, using the string template
html = HTML(link_t)

# display the HTML object to put the link on the page:
display(html)
